In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor, BitsAndBytesConfig
import torch
from qwen_vl_utils import process_vision_info


W0528 00:29:41.405000 40756 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Set False for 8-bit
    bnb_4bit_compute_dtype=torch.float16
)

In [3]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto", quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


In [5]:
image_path = "./Test_Images/FriedRice.jpeg"
dish = "Rice"

In [6]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]

In [7]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [8]:
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
description_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(description_text)

['The image depicts a plate of food placed on a light gray surface. The main dish appears to be a stir-fried rice with various ingredients. The rice is golden-yellow, indicating it has been cooked properly and possibly seasoned. Mixed into the rice are pieces of shrimp, which are pinkish-orange in color, suggesting they have been cooked or fried. Additionally, there are green onions sprinkled over the dish, adding a pop of color and likely a fresh, slightly crunchy texture.\n\nTo the left of the plate, there is a small white bowl containing a green garnish, possibly cilantro or parsley. To the right of the plate,']


In [9]:
recipe_generator = f"""You are a professional recipe generator.
Dish: {dish}
Description of the food : {description_text}

Format: The recipe should be in "3-steps" 

Examples: 
Dish: Mustard Fish
Summary: 1. Clean and slice the fish. Soak mustard seeds for 2 hours, then grind with salt and green chillies to a paste. Mix in grated coconut, yoghurt, sugar, turmeric, and mustard oil to make a marinade.
2. Coat the fish pieces with the mixture, place them in a tiffin box with halved green chillies on top, fasten the lid, and marinate for at least 15 minutes.
3. Steam the sealed tiffin in a covered pan with hot water for 15 minutes. Let it rest for 5 minutes before serving.

Dish: Butter Chicken
Summary: 1. Marinate boneless chicken pieces in yogurt, lemon juice, ginger-garlic paste, chili powder, and garam masala for at least 1 hour. Then grill, bake, or pan-fry until cooked and slightly charred.
2. In a pan, heat butter and sauté ginger, garlic, and tomatoes until soft. Blend this into a smooth puree, then return to the pan and add cream, kasuri methi (dried fenugreek), chili powder, and salt. Simmer until rich and creamy.
3. Add the cooked chicken to the sauce, simmer for 10–15 minutes, adjust seasoning, and serve hot with naan or rice.

Dish: Cheesy Bake
Summary: 1. Boil or steam your choice of veggies (e.g., broccoli, cauliflower, carrots, corn) or cooked pasta/chicken. Lightly season with salt, pepper, and herbs.
2. In a pan, melt butter, add flour, and cook for a minute. Slowly whisk in milk to make a smooth white sauce. Add grated cheese (like cheddar or mozzarella) and stir until melted and creamy.
3. Mix the base with the cheese sauce, pour into a greased baking dish, top with more cheese, and bake at 180°C (350°F) for 20–25 minutes until golden and bubbly. Serve hot!"""

In [10]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": image_path,
            },
            {"type": "text", "text": recipe_generator},
        ],
    }
]

In [14]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
).to("cuda")

In [15]:
generated_ids = model.generate(
    **inputs,
    max_new_tokens=300,
    ).to("cpu")
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
recipe_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(recipe_text)

['Dish: Fried Rice\nSummary: 1. Cook rice according to package instructions. In a large skillet, heat oil over medium-high heat. add chopped red bell peppers, green bell peppers, and onion. Stir-fry for 2-3 minutes until softened. Add shrimp and cook for another 2-3 minutes until lightly browned. Add rice and stir-fry for 1-2 minutes until heated through. Season with soy sauce, sesame oil, and any desired seasonings such as garlic, ginger, or chili flakes. Garnish with chopped green onions and serve hot.\n\n']
